# Check PSF correction in A360 field

Contact author: Céline Combet; with inputs of many during March 26 telecon (Anthony, Ian, Miranda, Shenming,...)\
LSST Science Piplines version: Weekly 2025_09\
Container Size: large

This notebook aims at check the PSF behaviour and correction in A360 field, that we use to perform the WL analysis. The main steps are

- Loading the relevant stars from the object catalogs (all tracts and patches needed) using the butler
- Checking out the size of the PSF accross the field
- Computing the ellipticities of stars and corresponding PSF model and make the whisker plots to check out the residuals. 

NB: All is done in x-y coordinates. We were puzzled how the radec selection of the stars in the fields translated into x-y (see 'Check location of PSF stars section below')

NB: Check out the [PSF DP0.2 tutorial](https://github.com/lsst/tutorial-notebooks/blob/main/DP0.2/12b_PSF_Science_Demo.ipynb) for more PSF diagnostics 

In [ ]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom
import lsst.afw.geom as afwGeom

In [ ]:
repo = '/repo/main'
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_09/DM-49235'
butler = Butler(repo, collections=collection)
skymap = butler.get('skyMap', skymap='lsst_cells_v1')

## Load the relevant catalogs
For PSF studies, we need to look at stars
### Find all tracts/patches to load

In [ ]:
# Position of the BCG for A360
ra_bcg = 37.862
dec_bcg = 6.98

# Looking for all patches in delta deg region around it
delta = 0.5
center = geom.SpherePoint(ra_bcg, dec_bcg, geom.degrees)
ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

ra_range = (ra_min, ra_max)
dec_range = (dec_min, dec_max)
radec = [geom.SpherePoint(ra_range[0], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[0], dec_range[1], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[1], geom.degrees)]

tracts_and_patches = skymap.findTractPatchList(radec)

tp_dict = {}
for tract_num in np.arange(len(tracts_and_patches)):
    tract_info = tracts_and_patches[tract_num][0]
    tract_idx = tract_info.getId()
    # All the patches around the cluster
    patches = []
    for i,patch in enumerate(tracts_and_patches[tract_num][1]):
        patch_info = tracts_and_patches[tract_num][1][i]
        patch_idx = patch_info.sequential_index
        patches.append(patch_idx)
    tp_dict.update({tract_idx:patches})
#tp_dict

### Load quantities with the cuts needed to get PSF stars, etc. 

In [ ]:
# Get the object catlaog of these patches
datasetType = 'objectTable'

merged_cat_used = pd.DataFrame() # to store the catalog of stars used by PIFF for the PSF modeling
merged_cat_reserved = pd.DataFrame() # to store the catalog of stars marked as "reserved", i.e. not used to build the PIFF PSF model 
merged_cat_all = pd.DataFrame() # to store all star-like objects, to have more locations to check the PSF model.

for tract in list(tp_dict.keys()):
    print(f'Loading objects from tract {tract}, patches:{tp_dict[tract]}')
    for patch in tp_dict[tract]:
#        print(patch)
        dataId = {'tract': tract, 'patch' : patch ,'skymap':'lsst_cells_v1'}
        obj_cat = butler.get(datasetType, dataId=dataId)

        # Stars used for the PSF modeling
        filt = obj_cat['detect_isPrimary'] == True
        filt &= obj_cat['refExtendedness'] == 0.0 # keep stars only
        filt &= obj_cat['i_calib_psf_used'] == True # that were used to build the psf model
        filt &= obj_cat['i_pixelFlags_inexact_psfCenter'] == False # To avoid objects with discontinuous PSF (due to edges)
        filt &= obj_cat['i_calibFlux'] > 360 # nJy, be bright
        merged_cat_used = pd.concat([merged_cat_used, obj_cat[filt]], ignore_index=True)

        # Stars "reserved" to check the PSF modeling
        filt = obj_cat['detect_isPrimary'] == True
        filt &= obj_cat['refExtendedness'] == 0.0
        filt &= obj_cat['i_calib_psf_reserved'] == True # not used for the psf model
        filt &= obj_cat['i_pixelFlags_inexact_psfCenter']==False
        merged_cat_reserved = pd.concat([merged_cat_reserved, obj_cat[filt]], ignore_index=True)

        # All extended objects (to have more locations where to look at the PSF model)
        filt = obj_cat['detect_isPrimary']==True
        filt &= obj_cat['refExtendedness'] == 1.0
        merged_cat_all = pd.concat([merged_cat_all, obj_cat[filt]], ignore_index=True)

### Check out the location of the PSF stars, in (ra, dec) and (x,y) coordinates, colored by track number - Reason for the pattern/gap in (x,y)?

The BCG and 0.5 deg field are highlighted in the (ra,dec) plot

In [ ]:
from matplotlib.patches import Circle

circle1 = Circle((ra_bcg, dec_bcg), 0.5, color='black', fill=False, linewidth=0.5)

color = ['red', 'blue','green','magenta']

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
for i,tract in enumerate(list(tp_dict.keys())):
    filt = merged_cat_used['tract'] == tract
    ax[0].scatter(merged_cat_used[filt]['coord_ra'], merged_cat_used[filt]['coord_dec'], 
                  c=color[i],  marker='.', s=2, label=f'tract = {tract}')
    
    ax[0].set_xlabel('ra [deg]')
    ax[0].set_ylabel('dec [deg]')
    ax[0].add_patch(circle1)

for i,tract in enumerate(list(tp_dict.keys())):
    filt = merged_cat_used['tract'] == tract
    ax[1].scatter(merged_cat_used[filt]['i_centroid_x'], merged_cat_used[filt]['i_centroid_y'],  marker='.', s=2, c=color[i])
ax[1].set_xlabel('i_centroid_x')
ax[1].set_ylabel('i_centroid_y')
fig.tight_layout()

ax[0].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='black')

fig.legend(loc=9, markerscale=10)

In (ra,dec), the stars cover the field and we can see which tract contribute to which area. In (x,y), we see a clear gap between the tracts. We do not understand where this is coming from (need to ask DM folks), but we can nontheless move on with the PSF charracterisation in the field.

## PSF size variation across the field

The `i_i{xx,xy,yy}PSF` quantities are the second moment of the PSF model for each object location in the catalog. The trace radius (PSF size) of the PSF is defined as
$r_t = \sqrt{(I_{xx} + I_{yy}/2)}$

We look at the size of the PSF:
- at the location of `used` stars (`merged_cat_used` catalog)
- at the location of all extended objects (`merged_cat_all` catalog), to have a better coverage of the field and visualize PSF discontinutities, etc.


In [ ]:
size = np.sqrt((merged_cat_used['i_ixxPSF'] + merged_cat_used['i_iyyPSF']) / 2)
size_all = np.sqrt((merged_cat_all['i_ixxPSF'] + merged_cat_all['i_iyyPSF']) / 2) # at all extended objects locations

In [ ]:
from matplotlib.patches import Circle

ra, dec =  merged_cat_used['coord_ra'], merged_cat_used['coord_dec']

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,5))

scatter_plot1 = ax[0].scatter(ra, dec, c=size, s=4, cmap='viridis', marker='o')
circle1 = Circle((ra_bcg, dec_bcg), 0.5, color='orange', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')

ax[0].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
ax[0].add_patch(circle1)
#ax.add_patch(circle2)

scatter_plot2 = ax[1].scatter(merged_cat_all['coord_ra'], merged_cat_all['coord_dec'], 
                              c=size_all, s=1, cmap='viridis', marker='o')
ax[1].set_xlabel('ra')
ax[1].set_ylabel('dec')
ax[1].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
circle2 = Circle((ra_bcg, dec_bcg), 0.5, color='orange', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')
ax[1].add_patch(circle2)

plt.colorbar(scatter_plot1, ax=ax[0], label='PSF size [pixels]')
plt.colorbar(scatter_plot1, ax=ax[1], label='PSF size [pixels]')

The PSF size is varying by ~0.5 pixel across A360 field. The figure on the right, showing the PSF size at more 
locations that highlight the discontinuities in the PSF modeling when close to edges. Also allows us to see the various orientation of visits used to build the coadd.

## Whisker plots - PSF ellipticity and PSF correction.

The ellipticity components $e_1$, $e_2$ are computed from moments as:

$e_1 = (I_{xx} - I_{yy}) / (I_{xx} + I_{yy})$

$e_2 = 2I_{xy} / (I_{xx} + I_{yy})$

The from this, the amplitude and orientation of the ellipse (angle of the ellipse major axis with respect to the (x,y) coordinate frame) are given by

$e = \sqrt{e_1^2 + e_2^2}$

$\theta = 0.5 \times \arctan (e_2/e_1)$

In [ ]:
def get_psf_ellip(catalog):
    psf_mxx = catalog['i_ixxPSF']
    psf_myy = catalog['i_iyyPSF']
    psf_mxy = catalog['i_ixyPSF']
    return (psf_mxx - psf_myy) / (psf_mxx + psf_myy), 2.* psf_mxy / (psf_mxx + psf_myy)


def get_star_ellip(catalog):
    star_mxx = catalog['i_ixx']
    star_myy = catalog['i_iyy']
    star_mxy = catalog['i_ixy']
    return (star_mxx - star_myy) / (star_mxx + star_myy), 2. * star_mxy / (star_mxx + star_myy)

    

### Whisker plot and residuals at the locations of `used` stars

In [ ]:
# For the PSF model, at the location of `used` stars
e1_psf_used, e2_psf_used = get_psf_ellip(merged_cat_used)
e_psf_used = np.sqrt(e1_psf_used*e1_psf_used + e2_psf_used*e2_psf_used) # module of ellipticity
theta_psf_used = 0.5 * np.arctan(e2_psf_used/e1_psf_used) # orientation

cx_psf_used = e_psf_used * np.cos(theta_psf_used) # x-component of the vector for the whisker plot
cy_psf_used = e_psf_used * np.sin(theta_psf_used) # y-component of the vector for the whisker plot

In [ ]:
# Repeat for the `used` stars
e1_star_used, e2_star_used = get_star_ellip(merged_cat_used)
e_star_used = np.sqrt(e1_star_used*e1_star_used+e2_star_used*e2_star_used)
theta_star_used = 0.5 * np.arctan(e2_star_used/e1_star_used)

cx_star_used = e_star_used * np.cos(theta_star_used)
cy_star_used = e_star_used * np.sin(theta_star_used)

In [ ]:
x_centroid, y_centroid =  merged_cat_used['i_centroid_x'], merged_cat_used['i_centroid_y']

scale = 10000

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
ax[0].quiver(x_centroid, y_centroid, scale*cx_star_used, scale*cy_star_used, angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
         label='Star ellipticity')
ax[0].set_title('Star ellipticity (from i_ixx, i_ixy, i_iyy)')
ax[0].set_xlabel('x_centroid')
ax[0].set_ylabel('y_centroid')

ax[1].quiver(x_centroid, y_centroid, scale*cx_psf_used, scale*cy_psf_used, angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
         label='PSF model ellipticity')
ax[1].set_xlabel('x_centroid')
ax[1].set_title('PSF model ellipticity (from i_ixxPSF, i_ixyPSF, i_iyyPSF)')

ax[2].quiver(x_centroid, y_centroid, scale*(cx_star_used-cx_psf_used), scale*(cy_star_used-cy_psf_used), angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0)

ax[2].set_xlabel('x_centroid')
ax[2].set_title('Star - PSF ellipticity residuals')
fig.tight_layout()


### Whisker plot and residuals at the locations of `reserved` stars (that haven't been used by PIFF)

Now we repeat the same thing with the `reserved` stars, that were not used to buid the PSF model. NB: there are far less reserved stars than PSF stars.

In [ ]:
e1_psf_reserved, e2_psf_reserved = get_psf_ellip(merged_cat_reserved)
e_psf_reserved = np.sqrt(e1_psf_reserved*e1_psf_reserved+e2_psf_reserved*e2_psf_reserved)
theta_psf_reserved = 0.5 * np.arctan(e2_psf_reserved/e1_psf_reserved)
cx_psf_reserved = e_psf_reserved * np.cos(theta_psf_reserved) # x-component of the vector for the whisker plot
cy_psf_reserved = e_psf_reserved * np.sin(theta_psf_reserved) # y-component of the vector for the whisker plot



e1_star_reserved, e2_star_reserved = get_star_ellip(merged_cat_reserved)
e_star_reserved = np.sqrt(e1_star_reserved*e1_star_reserved+e2_star_reserved*e2_star_reserved)
theta_star_reserved = 0.5 * np.arctan(e2_star_reserved/e1_star_reserved)
cx_star_reserved = e_star_reserved * np.cos(theta_star_reserved) # x-component of the vector for the whisker plot
cy_star_reserved = e_star_reserved * np.sin(theta_star_reserved) # y-component of the vector for the whisker plot

x_centroid, y_centroid =  merged_cat_reserved['i_centroid_x'], merged_cat_reserved['i_centroid_y']

scale = 10000

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
ax[0].quiver(x_centroid, y_centroid, scale*cx_star_reserved, scale*cy_star_reserved, angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
         label='PSF ellipticity')
ax[0].set_title('Star ellipticity (from i_ixx, i_ixy, i_iyy)')

ax[1].quiver(x_centroid, y_centroid, scale*cx_psf_reserved, scale*cy_psf_reserved, angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0,
         label='PSF ellipticity')
ax[1].set_title('PSF model ellipticity (from i_ixxPSF, i_ixyPSF, i_iyyPSF)')

ax[2].quiver(x_centroid, y_centroid, scale*(cx_star_reserved-cx_psf_reserved), scale*(cy_star_reserved-cy_psf_reserved), angles='xy', color='black',
           scale_units='xy', scale=1, headlength=0, headwidth=0, headaxislength=0)

ax[2].set_title('Star - PSF ellipticity residuals')


## Histogram of the e1, e2 residuals in A360 field, for `used` and `reserved` stars

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

ax[0].hist((e1_star_used-e1_psf_used), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='used');
ax[0].hist((e1_star_reserved-e1_psf_reserved), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='reserved')
ax[0].set_xlabel('Image e1 - PSF e1')
ax[0].set_xlabel(r'$\delta e_1 = $ Image e1 - PSF e1')
ax[0].legend()

ax[1].hist((e2_star_used-e2_psf_used), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='used');
ax[1].hist((e2_star_reserved-e2_psf_reserved), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='reserved')
ax[1].set_xlabel(r'$\delta e_2 = $ Image e2 - PSF e2')
ax[1].legend()